In [1]:
from __future__ import annotations

from datetime import datetime, timedelta
from typing import Callable, Dict, List, Optional, Tuple, Any, TypedDict

try:
    from langgraph.graph import StateGraph, END
except ImportError as e:
    raise ImportError(
        "langgraph is required for agent graph orchestration. Install with: pip install langgraph"
    ) from e

from schemas import (
    UserProfile,
    GitHubUserExtract,
    LinkedInProfileExtract,
    HuggingFaceModelExtract,
    CareerActionRecommendation,
)
from trend_scrapping_node import process_extractions_and_recommend
from User_interaction_node import run_interaction


# Optional pluggable callbacks (override from your app before building the graph)
RUN_USER_PROCESSING_CB: Optional[
    Callable[[UserProfile], Tuple[UserProfile, Optional[GitHubUserExtract], Optional[LinkedInProfileExtract], Optional[List[HuggingFaceModelExtract]]]]
] = None


class AgentState(TypedDict, total=False):
    user: UserProfile
    github: Optional[GitHubUserExtract]
    linkedin: Optional[LinkedInProfileExtract]
    hf_models: Optional[List[HuggingFaceModelExtract]]
    recs: List[CareerActionRecommendation]
    last_user_processing_at: Optional[datetime]
    next_user_processing_at: Optional[datetime]
    schedule_interval_days: int


# --- Nodes ---

def user_processing_node(state: AgentState) -> AgentState:
    user = state["user"]
    # Invoke custom ingestion if provided; else no-op (keep previous extracts)
    if RUN_USER_PROCESSING_CB is not None:
        user, github, linkedin, hf_models = RUN_USER_PROCESSING_CB(user)
        state["user"] = user
        state["github"] = github
        state["linkedin"] = linkedin
        state["hf_models"] = hf_models

    now = datetime.now()
    state["last_user_processing_at"] = now
    interval_days = int(state.get("schedule_interval_days", 7))
    state["next_user_processing_at"] = now + timedelta(days=interval_days)
    return state


def trend_scrapping_node(state: AgentState) -> AgentState:
    user, github, linkedin, hf_models = (
        state["user"],
        state.get("github"),
        state.get("linkedin"),
        state.get("hf_models"),
    )
    aligned_user, recs = process_extractions_and_recommend(user, github=github, linkedin=linkedin, hf_models=hf_models)
    state["user"] = aligned_user
    state["recs"] = recs
    return state


def user_interaction_node(state: AgentState) -> AgentState:
    user, github, linkedin, hf_models = (
        state["user"],
        state.get("github"),
        state.get("linkedin"),
        state.get("hf_models"),
    )
    # Reuse interaction flow; it internally handles approvals/tasks
    result = run_interaction(user, github_extract=github, linkedin_extract=linkedin, hf_models=hf_models)
    state["user"] = result["updated_profile"]
    # Keep recommendations as context (could be refreshed next cycle)
    state.setdefault("recs", [])
    return state


# --- Routing helpers ---

def route_after_interaction(state: AgentState) -> str:
    """Loop in interaction until the next scheduled processing time."""
    next_at = state.get("next_user_processing_at")
    now = datetime.now()
    if next_at and now >= next_at:
        return "user_processing"
    return "user_interaction"


# --- Graph factory ---

def build_agent(user: UserProfile, schedule_interval_days: int = 7):
    """Build and compile the LangGraph app with initial state returned.

    Returns: (app, initial_state)
    """
    sg = StateGraph(AgentState)
    sg.add_node("user_processing", user_processing_node)
    sg.add_node("trend_scrapping", trend_scrapping_node)
    sg.add_node("user_interaction", user_interaction_node)

    sg.set_entry_point("user_processing")
    sg.add_edge("user_processing", "trend_scrapping")
    sg.add_edge("trend_scrapping", "user_interaction")
    sg.add_conditional_edges(
        "user_interaction",
        route_after_interaction,
        {
            "user_processing": "user_processing",
            "user_interaction": "user_interaction",
        },
    )

    app = sg.compile()
    init_state: AgentState = {
        "user": user,
        "github": None,
        "linkedin": None,
        "hf_models": None,
        "recs": [],
        "last_user_processing_at": None,
        "next_user_processing_at": datetime.now(),  # run immediately
        "schedule_interval_days": schedule_interval_days,
    }
    return app, init_state


# Optional helper to step a limited number of times (to avoid infinite loops)
def run_steps(app, state: AgentState, max_steps: int = 3):
    for _ in range(max_steps):
        state = app.invoke(state)
    return state

In [2]:
from hard_coded_examples import (
    build_github_user_output_from_analysis,
    github_profile_analysis,
    build_linkedin_profile_output_from_parser,
    linked_in_profile_parser,
)
from User_Processing_node import User_report

# Build structured outputs from hard-coded examples
github_user_output = build_github_user_output_from_analysis(github_profile_analysis)
linked_in_user_output = build_linkedin_profile_output_from_parser(linked_in_profile_parser)

# Synthesize a unified UserProfile (now normalizes skills and certifications)
hard_coded_user_report = User_report(github_user_output, linked_in_user_output)

# Display the result
hard_coded_user_report

c:\Users\praty\Desktop\shortnote\resume projects\LakshSetu\LakshSetu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UserProfile(id=1, email='biswajitpolai5@gmail.com', name='biswajitpolai', age=None, location='Berhampur,odisha ,India', github=None, linkedin=None, huggingface=None, x=None, website=None, certifications=[Certifications(title='Big Data Modeling and Management Systems', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='Data Analytics in Python', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='ABV-IIITM Credentials', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='IIT BBS credentials', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='Python for Data science', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None)], skills=None, projects=[], blogs=None, achievements=["Runner's up at Hack For Tomorr

In [12]:
build_agent(hard_coded_user_report, schedule_interval_days=7)

(<langgraph.graph.state.CompiledStateGraph at 0x232968996a0>,
 {'user': UserProfile(id=1, email='biswajitpolai5@gmail.com', name='biswajitpolai', age=None, location='Berhampur,odisha ,India', github=None, linkedin=None, huggingface=None, x=None, website=None, certifications=[Certifications(title='Big Data Modeling and Management Systems', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='Data Analytics in Python', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='ABV-IIITM Credentials', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='IIT BBS credentials', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None), Certifications(title='Python for Data science', issuer='LinkedIn', issued_date='', credential_id=None, tags=[], certificate_strength=None)], skills=None